In [1]:
days_lookback = 5
pred_thresh =.65

In [2]:
import pickle

with open('ml_metadata_mr.pkl', 'rb') as ml_meta_file:
    ml_meta = pickle.load(ml_meta_file)

In [3]:
index_to_strat = {ml_meta['strategy_index'][x]: x for x in ml_meta['strategy_index']}

In [4]:
import os
import pandas as pd


def load_profits(fill_na_dict, strategy_index, feat_col, profit_folder='profit_reports/mean_reversion'):
    profit_files = [x for x in os.listdir(profit_folder) if x.endswith('.parquet')]
    all_cols = feat_col + ['symbol', 'date', 'norm_profit', 'actual_enter', 'actual_exit', 'exits', 'volatility_short']
    dfs = {x.split('_')[0]: pd.read_parquet('%s/%s' % (profit_folder, x))[all_cols] for x in profit_files}
    for key in dfs:
        dfs[key].loc[:, 'strategy_ind'] = strategy_index[key]
    df = pd.concat([dfs[x].fillna(fill_na_dict) for x in dfs], axis=0, ignore_index=True)
    return df

In [5]:
df = load_profits(ml_meta['fill_na_dict'], ml_meta['strategy_index'], ml_meta['feat_col'])

In [6]:
from joblib import load

with open('rfc_mr.joblib', 'rb') as rfc_file:
    rfc = load(rfc_file)

In [7]:
import datetime as dt
import numpy as np

max_date = df['date'].max()
min_date = max_date - dt.timedelta(days=days_lookback)
df_date_filt = df.loc[df['date'].between(min_date, max_date) & (df.roc_short != np.inf)]

In [8]:
preds = rfc.predict_proba(df_date_filt[ml_meta['feat_col']].fillna(0))[0][:, 1]

In [9]:
df_date_filt.loc[:, 'pred'] = 0
df_date_filt.loc[preds >= pred_thresh, 'pred'] = 1
df_date_filt.loc[:, 'pred_score'] = preds

/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
df_date_filt.loc[:, 'strategy'] = df_date_filt.strategy_ind.map(index_to_strat)

In [11]:
entries = df_date_filt.loc[df_date_filt.actual_enter == 1].sort_values(['date', 'pred', 'volatility'], ascending=[False, False, False])

In [12]:
exits = df_date_filt.loc[df_date_filt.exits == 1]

In [13]:
actual_exits = df_date_filt.loc[df_date_filt.actual_exit == 1]

In [14]:
cols = ['date', 'symbol', 'pred', 'pred_score', 'volatility', 'volatility_short', 'roc', 'rsi', 'mfi', 'sto', 'bba', 'dch', 'Volume','Close', 'strategy']

In [15]:
entries[cols].to_csv('signals/mean_reversion/entries.csv', index=False)

In [16]:
exits[cols].to_csv('signals/mean_reversion/exits.csv', index=False)

In [17]:
import pandas as pd

positions = pd.read_csv('signals/mean_reversion/mean_reversion_positions.csv', parse_dates=['date'])

In [18]:
actual_exits = positions.merge(exits, on=['symbol', 'strategy'], suffixes=["", "_y"])

In [19]:
actual_exits = actual_exits.loc[(actual_exits.date < actual_exits.date_y)]

In [20]:
actual_exits[cols].to_csv('signals/mean_reversion/actual_exits.csv', index=False)

In [21]:
num_positions = 2

In [22]:
entry_symbols = entries[['strategy', 'symbol']].drop_duplicates().iloc[:num_positions]

In [23]:
import os

def create_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [24]:
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots

def make_symbol_plot(sym_df, indicator, folder):
    sym_df = sym_df.sort_values('Date')
    fig = make_subplots(rows=2, cols=1, row_heights=[0.5, 0.5], shared_xaxes=True)

    fig.add_trace(
        go.Candlestick(
            x=sym_df.Date, open=sym_df.adj_open, high=sym_df.adj_high, low=sym_df.adj_low, close=sym_df.adj_close, name='bars'
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=sym_df.Date, y=sym_df[indicator], name=indicator, marker=dict(color='purple')
        ),
        row=2, col=1
    )
    fig.update_layout(xaxis_rangeslider_visible=False)
    plotly.offline.plot(fig, filename=f'{folder}/{sym_df.symbol.iloc[0]}.html', auto_open=False)

def plot_entry_symbols(entry_symbols, df, folder):
    create_dir(folder)
    for index in entry_symbols.index:
        symbol = entry_symbols.loc[index, 'symbol']
        strategy =  entry_symbols.loc[index, 'strategy']
        make_symbol_plot(df.loc[df.symbol == symbol], strategy.lower(), folder)


In [25]:
df = pd.read_parquet('ta_data.parquet')

In [26]:
folder = 'signals/mean_reversion/plots'
plot_entry_symbols(entry_symbols, df, folder)

In [27]:
#actual_exits[cols].to_csv('signals/actual_exits.csv', index=False)

positions = pd.read_excel('positions/positions.xlsx')

exits_positions = exits.merge(positions, right_on='position', left_on='symbol')

exits_positions.to_excel('signals/position_exits.xlsx', index=False)

In [28]:
import ibapp

In [29]:
def get_positions():
    positions_app = ibapp.PositionsApp()
    positions_app.connect('127.0.0.1',  7497, clientId=0)
    positions_app.get_positions()
    positions_app.run()
    return positions_app.positions_df.loc[positions_app.positions_df.position != 0]

In [30]:
positions = get_positions()

Position. Account: U3443642 Symbol: SPCE SecType: STK Currency: USD Position: 4.0 Avg cost: 25.76
Position. Account: U3443642 Symbol: RP SecType: STK Currency: USD Position: 1.0 Avg cost: 88.5568
Position. Account: U3443642 Symbol: TENB SecType: STK Currency: USD Position: 2.0 Avg cost: 52.85
Position. Account: U3443642 Symbol: MUR SecType: STK Currency: USD Position: 7.0 Avg cost: 12.2109
Position. Account: U3443642 Symbol: CAR SecType: STK Currency: USD Position: 3.0 Avg cost: 36.11333335
Position. Account: U3443642 Symbol: FEYE SecType: STK Currency: USD Position: 5.0 Avg cost: 18.2204
Position. Account: U3443642 Symbol: VER SecType: STK Currency: USD Position: 3.0 Avg cost: 36.31333335
Position. Account: U3443642 Symbol: FCEL SecType: STK Currency: USD Position: 9.0 Avg cost: 11.9184111
Position. Account: U3443642 Symbol: PACB SecType: STK Currency: USD Position: 4.0 Avg cost: 29.05
Position. Account: U3443642 Symbol: AJRD SecType: STK Currency: USD Position: 2.0 Avg cost: 52.84


In [31]:
exits.loc[exits.symbol.isin(positions.symbol.values)][cols].to_csv('signals/mean_reversion/position_exits.csv', index=False)